In [16]:
import random
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as Acivation
from sklearn.model_selection import train_test_split

from torch import nn
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/tdata.zip', 'r')
file = zip_ref.extractall("./tmp")
zip_ref.close()

print("o")

o


In [19]:
class CNN(nn.Module):

    def __init__(self,vgg_features):
        super().__init__()

        self.features = vgg_features

    def forward(self, z):
        z = self.features(z)
        z = z.view(-1, 512*7*7)

        return z


class CNN_LSTM(nn.Module):

    def __init__(self, vgg_features):
        super(CNN_LSTM, self).__init__()
        self.cnn = CNN(vgg_features)
        self.rnn = nn.LSTM(
            input_size=512*7*7, 
            hidden_size=64, 
            num_layers=1,
            batch_first=True)
        self.linear = nn.Linear(64,10)

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        c_in = x.view(batch_size * timesteps, C, H, W)
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, timesteps, -1)
        r_out, (h_n, h_c) = self.rnn(r_in)
        r_out2 = self.linear(r_out[:, -1, :])
        
        return F.log_softmax(r_out2, dim=1)

In [20]:
vgg_f =  models.vgg16(pretrained=True).features
model = CNN_LSTM(vgg_f)


In [21]:
# load model
PATH = '/content/drive/MyDrive/Colab Notebooks/CSE16/Luna/weights/lstm(e20).pth'
model = torch.load(PATH)
# print(model)

In [23]:
from PIL import Image

# compose allow us to chain multiple transform together
transform1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5,))

])

CATEGORIES=["Tomato___Bacterial_spot","Tomato___Early_blight",
            "Tomato___healthy","Tomato___Late_blight",
            "Tomato___Leaf_Mold","Tomato___Septoria_leaf_spot",
            "Tomato___Spider_mites Two-spotted_spider_mite","Tomato___Target_Spot",
            "Tomato___Tomato_mosaic_virus","Tomato___Tomato_Yellow_Leaf_Curl_Virus"]
        
path = '/content/tmp/tdata/valid/Tomato___Target_Spot/01f0e0cb-83a7-4190-9645-bd06b70c8d65___Com.G_TgS_FL 8318_new30degFlipLR.JPG'
img = Image.open(path)
# img.shape
img = transform1(img)
img.shape

image = img.view(1,1,img.shape[0], img.shape[1], img.shape[2])
image.shape


output = model(image.to(device)).to(device)
_,pred=torch.max(output,1)
print(pred[0].item())
print(CATEGORIES[pred[0].item()])

7
Tomato___Target_Spot
